In [1]:
import os
import gc
import re
import sys
import time
import json
import math
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

## 一.新款型取低配数据进行人工评估

In [46]:
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
add_combine_detail = pd.read_csv('../tmp/train/add_combine_detail.csv')

car_autohome_all = car_autohome_all.drop(['volume','control','volume_extend','emission_standard'], axis=1)
add_combine_detail = car_autohome_all.loc[(car_autohome_all['detail_slug'].isin(list(add_combine_detail.car_autohome_detail_id.values))), :].reset_index(drop=True)
add_combine_detail = add_combine_detail.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)

# 取低配数据
low_config_car = add_combine_detail.loc[add_combine_detail.groupby(['brand_slug', 'model_slug', 'online_year']).price_bn.idxmin(), :].reset_index(drop=True)
low_config_car['used_years'] = datetime.datetime.now().year - low_config_car['online_year']
low_config_car.loc[(low_config_car['used_years'] < 0), 'used_years'] = 0
low_config_car['median_price'] = np.NaN
low_config_car.to_csv('../tmp/train/global_model_mean_part3.csv', index=False)

In [47]:
global_model_mean_part3 = pd.read_csv('../tmp/train/global_model_mean_part3.csv')
global_model_mean_part3 = global_model_mean_part3.drop(['median_price'],axis=1)
global_model_mean_part3_copy = pd.read_csv('../tmp/train/global_model_mean_part3 (copy).csv')
global_model_mean_part3 = global_model_mean_part3.merge(global_model_mean_part3_copy.loc[:, ['detail_slug','median_price']],how='left',on=['detail_slug'])
global_model_mean_part3.to_csv('../tmp/train/global_model_mean_part3.csv', index=False)
global_model_mean_part3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 0 to 116
Data columns (total 13 columns):
detail_slug     117 non-null int64
brand_name      117 non-null object
brand_slug      117 non-null int64
model_name      117 non-null object
model_slug      117 non-null int64
detail_name     117 non-null object
manufacturer    117 non-null object
price_bn        117 non-null float64
body            117 non-null object
energy          117 non-null object
online_year     117 non-null int64
used_years      117 non-null int64
median_price    117 non-null float64
dtypes: float64(2), int64(5), object(6)
memory usage: 12.8+ KB


## 二.依据人工评估评估数据生成新款型估值

In [28]:
part1 = pd.read_csv('../tmp/train/global_model_mean.csv')
low_config_car = pd.read_csv('../tmp/train/global_model_mean_part3.csv')
low_config_car['median_price'] = low_config_car['median_price'] / 0.95
div_price_bn_k_param = pd.read_csv('../tmp/train/div_price_bn_k_param.csv')
combine_detail = pd.read_csv('../tmp/train/combine_detail.csv',low_memory=False)
add_combine_detail = pd.read_csv('../tmp/train/add_combine_detail.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.drop(['volume','control','volume_extend','emission_standard'], axis=1)

# car_autohome_all = car_autohome_all.loc[(car_autohome_all['detail_slug'].isin(list(add_combine_detail.car_autohome_detail_id.values))), :].reset_index(drop=True)
car_autohome_all = car_autohome_all.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)
car_autohome_all['used_years'] = datetime.datetime.now().year - car_autohome_all['online_year']
car_autohome_all.loc[(car_autohome_all['used_years'] < 0), 'used_years'] = 0

low_config_car = low_config_car.loc[(low_config_car['detail_slug'].isin(list(car_autohome_all.detail_slug.values))), :]

# 调整指导价差,确保同条件下高配比低配价格高
part2 = pd.DataFrame()
for model_slug, online_year in low_config_car.loc[:, ['model_slug', 'online_year']].values:
    car_autohome_temp = car_autohome_all.loc[(car_autohome_all['model_slug'] == model_slug)&(car_autohome_all['online_year'] == online_year), :].reset_index(drop=True)
    car_autohome_temp = car_autohome_temp.merge(low_config_car.loc[:, ['detail_slug', 'median_price']], how='left', on=['detail_slug'])
    low_config_price, price_bn = car_autohome_temp.loc[(car_autohome_temp['median_price'].notnull()), ['median_price', 'price_bn']].values[0]
    used_years = car_autohome_temp.loc[0, 'used_years']
    k = div_price_bn_k_param.loc[(div_price_bn_k_param['used_years'] == used_years), ['k']].values[0]
    for i in range(0, len(car_autohome_temp)):
        car_autohome_temp.loc[i, 'median_price'] = float('%.2f' % ((car_autohome_temp.loc[i, 'price_bn'] - price_bn) * k + low_config_price))
    part2 = part2.append(car_autohome_temp, sort=False).reset_index(drop=True)

print('原始款型数:', len(part1))
part1 = part1.loc[~(part1['detail_slug'].isin(list(set(add_combine_detail.car_autohome_detail_id.values)))), :]
part2 = part2.merge(combine_detail.loc[:, ['detail_model_slug', 'car_autohome_detail_id']].rename(columns={'car_autohome_detail_id':'detail_slug'}), how='left', on=['detail_slug'])
part2 = part2.loc[(part2['detail_model_slug'].notnull()), :]
global_model_mean = part1.append(part2, sort=False).reset_index(drop=True)
print('新增款型数:', len(add_combine_detail))
print('最终款型数:', len(global_model_mean), '实际应为:', len(combine_detail))
global_model_mean.to_csv('../tmp/train/global_model_mean.csv', index=False)

原始款型数: 34043
新增款型数: 787
最终款型数: 34582 实际应为: 34582


In [39]:
combine_brand = pd.read_csv('../tmp/train/combine_brand.csv',low_memory=False)
combine_brand = combine_brand.loc[:, ['name','slug']].rename(columns={'slug':'parent','name':'brand_name'})
combine_model = pd.read_csv('../tmp/train/combine_model.csv',low_memory=False)
combine_model = combine_model.merge(combine_brand,how='left',on=['parent'])
combine_model = combine_model.loc[:, ['slug','name','parent','brand_name']].rename(columns={'slug':'global_slug','name':'model_name','parent':'brand_slug'})
combine_detail = pd.read_csv('../tmp/train/combine_detail.csv',low_memory=False)
combine_detail = combine_detail.merge(combine_model,how='left',on=['global_slug'])
print('最终品牌数:',len(list(set(combine_detail.brand_slug.values))))
print('最终车系数:',len(list(set(combine_detail.global_slug.values))))
print('最终款型数:',len(list(set(combine_detail.detail_model_slug.values))))

最终品牌数: 182
最终车系数: 1851
最终款型数: 34582


In [40]:
global_model_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34582 entries, 0 to 34581
Data columns (total 15 columns):
detail_slug          34582 non-null int64
brand_name           34582 non-null object
brand_slug           34582 non-null int64
model_name           34582 non-null object
model_slug           34582 non-null int64
detail_name          34582 non-null object
manufacturer         34582 non-null object
price_bn             34582 non-null float64
body                 34582 non-null object
energy               34582 non-null object
online_year          34582 non-null int64
used_years           34582 non-null int64
median_price         34582 non-null float64
listed_year          11943 non-null float64
detail_model_slug    34582 non-null object
dtypes: float64(3), int64(5), object(7)
memory usage: 4.0+ MB
